# Machine Learning Model Architecture

## Construction of Test/Train datasets

In [9]:
import pandas as pd
# Let's make sure these directories are clean before we start
import shutil
try:
    shutil.rmtree("../data/project3/data_all_modified/data_split/train")
    shutil.rmtree("../data/project3/data_all_modified/data_split/test")
except:
    pass

In [10]:
# We have two classes which contains all the data: Damage & No_damage
# Let's create directories for each class in the train and test directories.
import os
# ensure directories exist
from pathlib import Path

Path("../data/project3/data_all_modified/data_split/test/damage").mkdir(parents=True, exist_ok=True)
Path("../data/project3/data_all_modified/data_split/test/no_damage").mkdir(parents=True, exist_ok=True)

Path("../data/project3/data_all_modified/data_split/train/damage").mkdir(parents=True, exist_ok=True)
Path("../data/project3/data_all_modified/data_split/train/no_damage").mkdir(parents=True, exist_ok=True)

In [11]:
# we need paths of images for individual classes so we can copy them in the new directories that we created above
all_damage_file_paths = os.listdir('../data/project3/data_all_modified/damage')
all_no_damage_file_paths = os.listdir('../data/project3/data_all_modified/no_damage')

In [12]:
import random

train_damage_paths = random.sample(all_damage_file_paths, int(len(all_damage_file_paths)*0.8))
print("train Damage image count: ", len(train_damage_paths))
test_damage_paths = [ p for p in all_damage_file_paths if p not in train_damage_paths]
print("test Damage image count: ", len(test_damage_paths))
# ensure no overlap:
overlap = [p for p in train_damage_paths if p in test_damage_paths]
print("len of overlap: ", len(overlap))

train_no_damage_paths = random.sample(all_no_damage_file_paths, int(len(all_no_damage_file_paths)*0.8))
print("train No Damage image count: ", len(train_no_damage_paths))
test_no_damage_paths = [ p for p in all_no_damage_file_paths if p not in train_no_damage_paths]
print("test No Damage image count: ", len(test_no_damage_paths))
# ensure no overlap:
overlap = [p for p in train_no_damage_paths if p in test_no_damage_paths]
print("len of overlap: ", len(overlap))


train Damage image count:  11336
test Damage image count:  2834
len of overlap:  0
train No Damage image count:  5721
test No Damage image count:  1431
len of overlap:  0


In [13]:
#ensure to copy the images to the directories
import shutil
for p in train_damage_paths:
    shutil.copyfile(os.path.join('../data/project3/data_all_modified/damage', p), os.path.join("../data/project3/data_all_modified/data_split/train/damage", p) )

for p in test_damage_paths:
    shutil.copyfile(os.path.join('../data/project3/data_all_modified/damage', p), os.path.join("../data/project3/data_all_modified/data_split/test/damage", p) )

for p in train_no_damage_paths:
    shutil.copyfile(os.path.join('../data/project3/data_all_modified/no_damage', p), os.path.join("../data/project3/data_all_modified/data_split/train/no_damage", p) )

for p in test_no_damage_paths:
    shutil.copyfile(os.path.join('../data/project3/data_all_modified/no_damage', p), os.path.join("../data/project3/data_all_modified/data_split/test/no_damage", p) )

# check counts:
print("Files in train/damage: ", len(os.listdir("../data/project3/data_all_modified/data_split/train/damage")))
print("Files in train/no_damage: ", len(os.listdir("../data/project3/data_all_modified/data_split/test/damage")))

print("Files in test/damage: ", len(os.listdir("../data/project3/data_all_modified/data_split/train/no_damage")))
print("Files in test/no_damage: ", len(os.listdir("../data/project3/data_all_modified/data_split/test/no_damage")))

Files in train/damage:  11336
Files in train/no_damage:  2834
Files in test/damage:  5721
Files in test/no_damage:  1431


## Data Pre-Processing

In [14]:
# Using PIL to get image dimensions
from PIL import Image

def get_image_dimensions(image_path: str):
    '''
    Gets image dimensions through the input of a directory

    Input: Takes string input of the directory path to an image

    Output: Returns image dimensions of height & width pixels
    '''
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            return width, height
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

def check_images_same_size(directory: str):
    '''
    Ensures that all images within a directory are the same size

    Input: Takes a string input of the desired directory to be check

    Output: Prints a statement stating images are the same size & their dimensions,
            or prints that the images have different sizes, or that there werent any valid images in
            the directory.
    '''
    dimensions_set = set()  # To store unique dimensions of images
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg"):
            image_path = os.path.join(directory, filename)
            dimensions = get_image_dimensions(image_path)
            if dimensions:
                dimensions_set.add(dimensions)

    if len(dimensions_set) == 1:
        dimensions = dimensions_set.pop()
        print("All images are the same size.")
        print(f"Image dimensions: {dimensions[0]}x{dimensions[1]} pixels")
    elif len(dimensions_set) > 1:
        print("Images have different sizes.")
    else:
        print("No valid images found in the directory.")

In [15]:
train_data_dmg = '../data/project3/data_all_modified/data_split/train/damage'
train_data_no_dmg = '../data/project3/data_all_modified/data_split/train/no_damage'
check_images_same_size(train_data_dmg)
check_images_same_size(train_data_no_dmg)

All images are the same size.
Image dimensions: 128x128 pixels
All images are the same size.
Image dimensions: 128x128 pixels


In [16]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
# Now that we know the image dimensions
train_data_dir = '../data/project3/data_all_modified/data_split/train'
# Number of images we want to process at once
batch_size = 64

# Target image size (128 px by 128 px)
img_height = 128
img_width = 128
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.


In [17]:
test_data_dir = '../data/project3/data_all_modified/data_split/train'

batch_size = 2

img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 17057 files belonging to 2 classes.


## A Dense ANN Model

In [18]:
# Building a CNN with 3 alternating convolutional layers & pooling layers
# with 2 dense hidden layers. Output layer has 3 classes & softmax activation

from keras import layers, models, optimizers
import pandas as pd

# initialize sequential model
model_cnn = models.Sequential()

# Convolutional layer with 64 filters, and a kernel size of 3x3.
# padding = 'same' gives the same output size as input_shape
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=(img_height,img_width,3)))

# Adding max pooling to reduce the size of output of first conv layer
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# Second Convolutional layer with Pooling layer to reduce size
model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# Final convolutional layer & pooling to reduce size
model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# flattening the output of the conv layer after max pooling 
# makes it ready for creating dense connections
model_cnn.add(layers.Flatten())

# Adding a fully connected dense layer with 100 neurons
model_cnn.add(layers.Dense(100, activation='relu'))

# Adding a fully connected dense layer with 84 neurons
model_cnn.add(layers.Dense(84, activation='relu'))

# Adding the output layer with 2 neurons and 
# activation functions as softmax since this is a multi-class classification problem
model_cnn.add(layers.Dense(2, activation='softmax'))

In [19]:
# Compile model
# RMSprop (Root Mean Square Propagation) is commonly used in training deep neural networks.
model_cnn.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

# Generating the summary of the model
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 32)        0

In [20]:
#fit the model from image generator
history = model_cnn.fit(
            train_rescale_ds,
            batch_size=64,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
214/214 [==============================] - 119s 553ms/step - loss: 0.6044 - accuracy: 0.6987 - val_loss: 0.6556 - val_accuracy: 0.6719
Epoch 2/20
214/214 [==============================] - 114s 529ms/step - loss: 0.4851 - accuracy: 0.7848 - val_loss: 0.4314 - val_accuracy: 0.8004
Epoch 3/20
214/214 [==============================] - 116s 540ms/step - loss: 0.4319 - accuracy: 0.8197 - val_loss: 0.3953 - val_accuracy: 0.8549
Epoch 4/20
214/214 [==============================] - 115s 537ms/step - loss: 0.3941 - accuracy: 0.8389 - val_loss: 0.4658 - val_accuracy: 0.7819
Epoch 5/20
214/214 [==============================] - 114s 532ms/step - loss: 0.3634 - accuracy: 0.8558 - val_loss: 0.3325 - val_accuracy: 0.8751
Epoch 6/20
214/214 [==============================] - 113s 526ms/step - loss: 0.3169 - accuracy: 0.8742 - val_loss: 0.3760 - val_accuracy: 0.8358
Epoch 7/20
214/214 [==============================] - 112s 525ms/step - loss: 0.2692 - accuracy: 0.8970 - val_loss: 0.2240 -

In [21]:
test_loss, test_accuracy = model_cnn.evaluate(test_rescale_ds, verbose = 0)
# validation accuracy
print(f'Test Loss: {test_loss}')
# test accuracy
print(f'Test Accuracy: {test_accuracy}')

Test Loss: 0.06554103642702103
Test Accuracy: 0.9777804017066956


## LeNet-5 Architecture

In [23]:
model_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 2 )
model_lenet5.add(layers.Dense(2, activation='softmax'))

# Compile model
model_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), 
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 6)       168       
                                                                 
 average_pooling2d (Average  (None, 63, 63, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 16)        880       
                                                                 
 average_pooling2d_1 (Avera  (None, 30, 30, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_1 (Flatten)         (None, 14400)             0         
                                                                 
 dense_3 (Dense)             (None, 120)              

In [24]:
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=64,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
214/214 [==============================] - 22s 100ms/step - loss: 0.6177 - accuracy: 0.6859 - val_loss: 0.5908 - val_accuracy: 0.7435
Epoch 2/20
214/214 [==============================] - 21s 99ms/step - loss: 0.5120 - accuracy: 0.7739 - val_loss: 0.5355 - val_accuracy: 0.7247
Epoch 3/20
214/214 [==============================] - 21s 100ms/step - loss: 0.4563 - accuracy: 0.8081 - val_loss: 0.5877 - val_accuracy: 0.7218
Epoch 4/20
214/214 [==============================] - 21s 98ms/step - loss: 0.4204 - accuracy: 0.8320 - val_loss: 0.3947 - val_accuracy: 0.8429
Epoch 5/20
214/214 [==============================] - 21s 99ms/step - loss: 0.3965 - accuracy: 0.8405 - val_loss: 0.3848 - val_accuracy: 0.8561
Epoch 6/20
214/214 [==============================] - 22s 105ms/step - loss: 0.3784 - accuracy: 0.8463 - val_loss: 0.3695 - val_accuracy: 0.8478
Epoch 7/20
214/214 [==============================] - 22s 101ms/step - loss: 0.3536 - accuracy: 0.8593 - val_loss: 0.3498 - val_accur

In [25]:
test_loss, test_accuracy = model_lenet5.evaluate(test_rescale_ds, verbose = 0)
# validation accuracy
print(f'Test Loss: {test_loss}')
# test accuracy
print(f'Test Accuracy: {test_accuracy}')

Test Loss: 0.17372162640094757
Test Accuracy: 0.931934118270874
